In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import time
import numpy as np
import tensorflow as tf

# VGG_MEAN = [123.68, 116.779, 103.939] # [R, G, B]
VGG_MEAN = [103.939, 116.779, 123.68] # [R, G, B]
class VGG16:
    def __init__(self, vgg16_npy_path=None):
        """
        load pre-trained weights from path
        :param vgg16_npy_path: file path of vgg16 pre-trained weights
        """
        
        # load pre-trained weights
        if vgg16_npy_path is None:
            path = inspect.getfile(Vgg16)
            path = os.path.abspath(os.path.join(path, os.pardir))
            vgg16_npy_path = os.path.join(path, "vgg16.npy")
            print(vgg16_npy_path)     
        self.data_dict = np.load(vgg16_npy_path,encoding='latin1').item()
        print("npy file loaded")
        
        # input information
        self.H, self.W, self.C = 32, 32, 3
        self.classes = 10
        
        # operation dictionary
        self.prob_dict = {}
        self.loss_dict = {}
        self.accu_dict = {}
        
        # input placeholder
        self.x = tf.placeholder(tf.float32, [None, self.H, self.W, self.C])
        self.y = tf.placeholder(tf.float32, [None, self.classes])

    def build(self, dp, prof_type):
        """
        load variable from npy to build the VGG
        :param rgb: rgb image [batch, height, width, 3] values scaled [0, 1]
        """
        
        self.dp = dp 
        print("Will optimize at DP=", self.dp)
        start_time = time.time()
        print("build model started")
        rgb_scaled = self.x * 255.0

        # normalize input by VGG_MEAN
        red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=rgb_scaled)
        assert   red.get_shape().as_list()[1:] == [self.H, self.W, 1]
        assert green.get_shape().as_list()[1:] == [self.H, self.W, 1]
        assert  blue.get_shape().as_list()[1:] == [self.H, self.W, 1]
        self.x = tf.concat(axis=3, values=[
              blue - VGG_MEAN[0],
            green - VGG_MEAN[1],
             red - VGG_MEAN[2],
        ])
        assert self.x.get_shape().as_list()[1:] == [self.H, self.W, self.C]
        
        # declare and initialize the weights of VGG16
        with tf.variable_scope("VGG16"):
        
            self.conv1_1_W, self.conv1_1_b = self.get_conv_filter("conv1_1"), self.get_bias("conv1_1")
            self.conv1_2_W, self.conv1_2_b = self.get_conv_filter("conv1_2"), self.get_bias("conv1_2")

            self.conv2_1_W, self.conv2_1_b = self.get_conv_filter("conv2_1"), self.get_bias("conv2_1")
            self.conv2_2_W, self.conv2_2_b = self.get_conv_filter("conv2_2"), self.get_bias("conv2_2")

            self.conv3_1_W, self.conv3_1_b = self.get_conv_filter("conv3_1"), self.get_bias("conv3_1")
            self.conv3_2_W, self.conv3_2_b = self.get_conv_filter("conv3_2"), self.get_bias("conv3_2")
            self.conv3_3_W, self.conv3_3_b = self.get_conv_filter("conv3_3"), self.get_bias("conv3_3")

            self.conv4_1_W, self.conv4_1_b = self.get_conv_filter("conv4_1"), self.get_bias("conv4_1")
            self.conv4_2_W, self.conv4_2_b = self.get_conv_filter("conv4_2"), self.get_bias("conv4_2")
            self.conv4_3_W, self.conv4_3_b = self.get_conv_filter("conv4_3"), self.get_bias("conv4_3")

            self.conv5_1_W, self.conv5_1_b = self.get_conv_filter("conv5_1"), self.get_bias("conv5_1")
            self.conv5_2_W, self.conv5_2_b = self.get_conv_filter("conv5_2"), self.get_bias("conv5_2")
            self.conv5_3_W, self.conv5_3_b = self.get_conv_filter("conv5_3"), self.get_bias("conv5_3")

            self.fc_1_W = tf.get_variable(name="fc_1_W", shape=(512, 512), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)
            self.fc_1_b = tf.get_variable(name="fc_1_b", shape=(512), initializer=tf.ones_initializer(), dtype=tf.float32)

            self.fc_2_W = tf.get_variable(name="fc_2_W", shape=(512, 10), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)
            self.fc_2_b = tf.get_variable(name="fc_2_b", shape=(10), initializer=tf.ones_initializer(), dtype=tf.float32)
        
        # create operations at every dot product percentages
        for dp_i in dp:
            with tf.name_scope(str(int(dp_i*100))):
                conv1_1 = self.idp_conv_layer( self.x, "conv1_1", dp_i, prof_type, gamma_trainable=False)
                conv1_2 = self.idp_conv_layer(conv1_1, "conv1_2", dp_i, prof_type, gamma_trainable=False)
                pool1 = self.max_pool(conv1_2, 'pool1')

                conv2_1 = self.idp_conv_layer(  pool1, "conv2_1", dp_i, prof_type, gamma_trainable=False)
                conv2_2 = self.idp_conv_layer(conv2_1, "conv2_2", dp_i, prof_type, gamma_trainable=False)
                pool2 = self.max_pool(conv2_2, 'pool2')

                conv3_1 = self.idp_conv_layer(  pool2, "conv3_1", dp_i, prof_type, gamma_trainable=False)
                conv3_2 = self.idp_conv_layer(conv3_1, "conv3_2", dp_i, prof_type, gamma_trainable=False)
                conv3_3 = self.idp_conv_layer(conv3_2, "conv3_3", dp_i, prof_type, gamma_trainable=False)
                pool3 = self.max_pool(conv3_3, 'pool3')

                conv4_1 = self.idp_conv_layer(  pool3, "conv4_1", dp_i, prof_type, gamma_trainable=False)
                conv4_2 = self.idp_conv_layer(conv4_1, "conv4_2", dp_i, prof_type, gamma_trainable=False)
                conv4_3 = self.idp_conv_layer(conv4_2, "conv4_3", dp_i, prof_type, gamma_trainable=False)
                pool4 = self.max_pool(conv4_3, 'pool4')

                conv5_1 = self.idp_conv_layer(  pool4, "conv5_1", dp_i, prof_type, gamma_trainable=False)
                conv5_2 = self.idp_conv_layer(conv5_1, "conv5_2", dp_i, prof_type, gamma_trainable=False)
                conv5_3 = self.idp_conv_layer(conv5_2, "conv5_3", dp_i, prof_type, gamma_trainable=False)
                pool5 = self.max_pool(conv5_3, 'pool5')

                fc_1 = self.fc_layer(pool5, 'fc_1')
                fc_1 = tf.nn.dropout(fc_1, keep_prob=0.5)
                fc_1 = tf.nn.relu(fc_1)
                
                logits = tf.nn.bias_add(tf.matmul( fc_1, self.fc_2_W), self.fc_2_b)
                prob = tf.nn.softmax(logits, name="prob")
                
                cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.y)
                loss = tf.reduce_mean(cross_entropy)
                accuracy = tf.reduce_mean(tf.cast(tf.equal(x=tf.argmax(logits, 1), y=tf.argmax(self.y, 1)),tf.float32))
                
                self.prob_dict[str(int(dp_i*100))] = prob
                self.loss_dict[str(int(dp_i*100))] = loss
                self.accu_dict[str(int(dp_i*100))] = accuracy
                
                tf.summary.scalar(name="accu_at_"+str(int(dp_i*100)), tensor=accuracy)
                tf.summary.scalar(name="loss_at_"+str(int(dp_i*100)), tensor=loss)
        
        self.summary_op = tf.summary.merge_all()
        print(("build model finished: %ds" % (time.time() - start_time)))

    def avg_pool(self, bottom, name):
        return tf.nn.avg_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def max_pool(self, bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def idp_conv_layer(self, bottom, name, dp, prof_type, gamma_trainable = False):
        with tf.name_scope(name+str(int(dp*100))):
            with tf.variable_scope("vgg16",reuse=True):
                conv_filter = tf.get_variable(name=name+"_W")
                conv_biases = tf.get_variable(name=name+"_b")
            
            H,W,C,O = conv_filter.get_shape().as_list()
        
            # get profile
            profile = self.get_profile(O, prof_type)
            
            # create a mask determined by the dot product percentage
            n1 = int(O * dp)
            n0 = O - n1
            mask = np.append(np.ones(n1, dtype='float32'), np.zeros(n0, dtype='float32'))
            if len(profile) == len(mask):
                profile *= mask
            else:
                raise ValueError("profile and mask must have the same shape.")

            # create a profile coefficient, gamma
            filter_profile = np.stack([profile for i in range(H*W*C)])
            filter_profile = np.reshape(filter_profile, newshape=(H, W, C, O))
            
            # gamma in use
            gamma_W = tf.Variable(initial_value=filter_profile, name=name+"_gamma_W_"+str(int(dp*100)), trainable=gamma_trainable)
            gamma_b = tf.Variable(initial_value=profile, name=name+"_gamma_W_"+str(int(dp*100)), trainable=gamma_trainable)

            # IDP conv2d output
            conv_filter = tf.multiply(conv_filter, gamma_W)
            conv_biases = tf.multiply(conv_biases, gamma_b)
            
            conv = tf.nn.conv2d(bottom, conv_filter, [1, 1, 1, 1], padding='SAME')
            conv = tf.nn.bias_add(conv, conv_biases)
            relu = tf.nn.relu(conv)
            
            return relu

    def fc_layer(self, bottom, name):
        with tf.name_scope(name):
            shape = bottom.get_shape().as_list()
            dim = 1
            for d in shape[1:]:
                dim *= d
            x = tf.reshape(bottom, [-1, dim])
            
            with tf.variable_scope("vgg16",reuse=True):
                weights = tf.get_variable(name=name+"_W")
                biases = tf.get_variable(name=name+"_b")

            # Fully connected layer. Note that the '+' operation automatically
            # broadcasts the biases.
            fc = tf.nn.bias_add(tf.matmul(x, weights), biases)
            return fc

    def get_conv_filter(self, name):
        return tf.get_variable(initializer=self.data_dict[name][0], name=name+"_W")
    def get_bias(self, name):
        return tf.get_variable(initializer=self.data_dict[name][1], name=name+"_b")
#     def get_fc_weight(self, name):
#         return tf.get_variable(initializer=self.data_dict[name][0], name=name+"_W")

    def get_profile(self, C, prof_type):
        def half_exp(n, k=1, dtype='float32'):
            n_ones = int(n/2)
            n_other = n - n_ones
            return np.append(np.ones(n_ones, dtype=dtype), np.exp((1-k)*np.arange(n_other), dtype=dtype))
        if prof_type == "linear":
            profile = np.linspace(1.0,0.0, num=C, endpoint=False, dtype='float32')
        elif prof_type == "all-one":
            profile = np.ones(C, dtype='float32')
        elif prof_type == "half-exp":
            profile = half_exp(C, 2.0)
        elif prof_type == "harmonic":
            profile = np.array(1.0/(np.arange(C)+1))
        else:
            raise ValueError("prof_type must be \"all-one\", \"half-exp\", \"harmonic\" or \"linear\".")
        return profile
                

# OLD

In [ ]:
import time
import numpy as np
import tensorflow as tf

VGG_MEAN = [123.68, 116.779, 103.939] # [R, G, B]

class VGG16:
    def __init__(self, vgg16_npy_path=None):
        """
        load pre-trained weights from path
        :param vgg16_npy_path: file path of vgg16 pre-trained weights
        """
        
        # load pre-trained weights
        if vgg16_npy_path is None:
            path = inspect.getfile(Vgg16)
            path = os.path.abspath(os.path.join(path, os.pardir))
            path = os.path.join(path, "vgg16_weights.npz")
            vgg16_npy_path = path
            print(path)     
        self.data_dict = np.load(vgg16_npy_path)
        print("npy file loaded")
        
        # input information
        self.H, self.W, self.C = 32, 32, 3
        self.classes = 10
        
        # operation dictionary
        self.prob_dict = {}
        self.loss_dict = {}
        self.accu_dict = {}
        
        # input placeholder
        self.x = tf.placeholder(tf.float32, [None, self.H, self.W, self.C])
        self.y = tf.placeholder(tf.float32, [None, self.classes])

    def build(self, dp, prof_type):
        """
        load variable from npy to build the VGG
        :param rgb: rgb image [batch, height, width, 3] values scaled [0, 1]
        """
        
        self.dp = dp 
        print("Will optimize at DP=", self.dp)
        start_time = time.time()
        print("build model started")
        self.x = self.x * 255.0

        # normalize input by VGG_MEAN
#         red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=rgb_scaled)
#         assert   red.get_shape().as_list()[1:] == [self.H, self.W, 1]
#         assert green.get_shape().as_list()[1:] == [self.H, self.W, 1]
#         assert  blue.get_shape().as_list()[1:] == [self.H, self.W, 1]
#         self.x = tf.concat(axis=3, values=[
#               blue - VGG_MEAN[0],
#             green - VGG_MEAN[1],
#              red - VGG_MEAN[2],
#         ])
        assert self.x.get_shape().as_list()[1:] == [self.H, self.W, self.C]
        mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1,1,1,3], name='img_mean')
        self.x = self.x - mean
        
        # declare and initialize the weights of VGG16
        with tf.variable_scope("vgg16"):
        
            self.conv1_1_W, self.conv1_1_b = self.get_conv_filter("conv1_1"), self.get_bias("conv1_1")
            self.conv1_2_W, self.conv1_2_b = self.get_conv_filter("conv1_2"), self.get_bias("conv1_2")

            self.conv2_1_W, self.conv2_1_b = self.get_conv_filter("conv2_1"), self.get_bias("conv2_1")
            self.conv2_2_W, self.conv2_2_b = self.get_conv_filter("conv2_2"), self.get_bias("conv2_2")

            self.conv3_1_W, self.conv3_1_b = self.get_conv_filter("conv3_1"), self.get_bias("conv3_1")
            self.conv3_2_W, self.conv3_2_b = self.get_conv_filter("conv3_2"), self.get_bias("conv3_2")
            self.conv3_3_W, self.conv3_3_b = self.get_conv_filter("conv3_3"), self.get_bias("conv3_3")

            self.conv4_1_W, self.conv4_1_b = self.get_conv_filter("conv4_1"), self.get_bias("conv4_1")
            self.conv4_2_W, self.conv4_2_b = self.get_conv_filter("conv4_2"), self.get_bias("conv4_2")
            self.conv4_3_W, self.conv4_3_b = self.get_conv_filter("conv4_3"), self.get_bias("conv4_3")

            self.conv5_1_W, self.conv5_1_b = self.get_conv_filter("conv5_1"), self.get_bias("conv5_1")
            self.conv5_2_W, self.conv5_2_b = self.get_conv_filter("conv5_2"), self.get_bias("conv5_2")
            self.conv5_3_W, self.conv5_3_b = self.get_conv_filter("conv5_3"), self.get_bias("conv5_3")

            self.fc_1_W = tf.get_variable(name="fc_1_W", shape=(512, 512), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)
            self.fc_1_b = tf.get_variable(name="fc_1_b", shape=(512), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)

            self.fc_2_W = tf.get_variable(name="fc_2_W", shape=(512, 512), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)
            self.fc_2_b = tf.get_variable(name="fc_2_b", shape=(512), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)

            self.fc_3_W = tf.get_variable(name="fc_3_W", shape=(512, 10), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)
            self.fc_3_b = tf.get_variable(name="fc_3_b", shape=(10), initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), dtype=tf.float32)

        
        # create operations at every dot product percentages
        for dp_i in dp:
            with tf.name_scope(str(int(dp_i*100))):
                conv1_1 = self.idp_conv_layer( self.x, "conv1_1", dp_i, prof_type, gamma_trainable=False)
                conv1_2 = self.idp_conv_layer(conv1_1, "conv1_2", dp_i, prof_type, gamma_trainable=False)
                pool1 = self.max_pool(conv1_2, 'pool1')

                conv2_1 = self.idp_conv_layer(  pool1, "conv2_1", dp_i, prof_type, gamma_trainable=False)
                conv2_2 = self.idp_conv_layer(conv2_1, "conv2_2", dp_i, prof_type, gamma_trainable=False)
                pool2 = self.max_pool(conv2_2, 'pool2')

                conv3_1 = self.idp_conv_layer(  pool2, "conv3_1", dp_i, prof_type, gamma_trainable=False)
                conv3_2 = self.idp_conv_layer(conv3_1, "conv3_2", dp_i, prof_type, gamma_trainable=False)
                conv3_3 = self.idp_conv_layer(conv3_2, "conv3_3", dp_i, prof_type, gamma_trainable=False)
                pool3 = self.max_pool(conv3_3, 'pool3')

                conv4_1 = self.idp_conv_layer(  pool3, "conv4_1", dp_i, prof_type, gamma_trainable=False)
                conv4_2 = self.idp_conv_layer(conv4_1, "conv4_2", dp_i, prof_type, gamma_trainable=False)
                conv4_3 = self.idp_conv_layer(conv4_2, "conv4_3", dp_i, prof_type, gamma_trainable=False)
                pool4 = self.max_pool(conv4_3, 'pool4')

                conv5_1 = self.idp_conv_layer(  pool4, "conv5_1", dp_i, prof_type, gamma_trainable=False)
                conv5_2 = self.idp_conv_layer(conv5_1, "conv5_2", dp_i, prof_type, gamma_trainable=False)
                conv5_3 = self.idp_conv_layer(conv5_2, "conv5_3", dp_i, prof_type, gamma_trainable=False)
                pool5 = self.max_pool(conv5_3, 'pool5')

#                 fc_1 = self.fc_layer(pool5, 'fc_1')
#                 fc_1 = tf.nn.dropout(fc_1, keep_prob=0.5)
#                 fc_1 = tf.nn.relu(fc_1)
#                 fc_2 = self.fc_layer(fc_1, 'fc_2')
                fc_2 = self.fc_layer(pool5, 'fc_2')
                
                fc_2 = tf.nn.dropout(fc_2, keep_prob=0.5)
                fc_2 = tf.nn.relu(fc_2)
                
                logits = tf.nn.bias_add(tf.matmul( fc_2, self.fc_3_W), self.fc_3_b)
                prob = tf.nn.softmax(logits, name="prob")
                
                cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.y)
                loss = tf.reduce_sum(cross_entropy)
                accuracy = tf.reduce_mean(tf.cast(tf.equal(x=tf.argmax(prob, 1), y=tf.argmax(self.y, 1)),tf.float32))
                
                self.prob_dict[str(int(dp_i*100))] = prob
                self.loss_dict[str(int(dp_i*100))] = loss
                self.accu_dict[str(int(dp_i*100))] = accuracy
                
                tf.summary.scalar(name="accu_at_"+str(int(dp_i*100)), tensor=accuracy)
                tf.summary.scalar(name="loss_at_"+str(int(dp_i*100)), tensor=loss)
        
        self.summary_op = tf.summary.merge_all()
        print(("build model finished: %ds" % (time.time() - start_time)))

    def avg_pool(self, bottom, name):
        return tf.nn.avg_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def max_pool(self, bottom, name):
        return tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

    def idp_conv_layer(self, bottom, name, dp, prof_type, gamma_trainable = False):
        with tf.name_scope(name+str(int(dp*100))):
            with tf.variable_scope("vgg16",reuse=True):
                conv_filter = tf.get_variable(name=name+"_W")
                conv_biases = tf.get_variable(name=name+"_b")
            
            H,W,C,O = conv_filter.get_shape().as_list()
        
            # get profile
            profile = self.get_profile(O, prof_type)
            
            # create a mask determined by the dot product percentage
            n1 = int(O * dp)
            n0 = O - n1
            mask = np.append(np.ones(n1, dtype='float32'), np.zeros(n0, dtype='float32'))
            if len(profile) == len(mask):
                profile *= mask
            else:
                raise ValueError("profile and mask must have the same shape.")

            # create a profile coefficient, gamma
            filter_profile = np.stack([profile for i in range(H*W*C)])
            filter_profile = np.reshape(filter_profile, newshape=(H, W, C, O))
            
            # gamma in use
            gamma_W = tf.Variable(initial_value=filter_profile, name=name+"_gamma_W_"+str(int(dp*100)), trainable=gamma_trainable)
            gamma_b = tf.Variable(initial_value=profile, name=name+"_gamma_W_"+str(int(dp*100)), trainable=gamma_trainable)

            # IDP conv2d output
            conv_filter = tf.multiply(conv_filter, gamma_W)
            conv_biases = tf.multiply(conv_biases, gamma_b)
            
            conv = tf.nn.conv2d(bottom, conv_filter, [1, 1, 1, 1], padding='SAME')
            conv = tf.nn.bias_add(conv, conv_biases)
            relu = tf.nn.relu(conv)
            
            return relu

    def fc_layer(self, bottom, name):
        with tf.name_scope(name):
            shape = bottom.get_shape().as_list()
            dim = 1
            for d in shape[1:]:
                dim *= d
            x = tf.reshape(bottom, [-1, dim])
            
            with tf.variable_scope("vgg16",reuse=True):
                weights = tf.get_variable(name=name+"_W")
                biases = tf.get_variable(name=name+"_b")

            # Fully connected layer. Note that the '+' operation automatically
            # broadcasts the biases.
            fc = tf.nn.bias_add(tf.matmul(x, weights), biases)
            return fc

    def get_conv_filter(self, name):
        return tf.get_variable(initializer=self.data_dict[name+"_W"], name=name+"_W")
    def get_bias(self, name):
        return tf.get_variable(initializer=self.data_dict[name+"_b"], name=name+"_b")
    def get_fc_weight(self, name):
        return tf.get_variable(initializer=self.data_dict[name+"_W"], name=name+"_W")

    def get_profile(self, C, prof_type):
        def half_exp(n, k=1, dtype='float32'):
            n_ones = int(n/2)
            n_other = n - n_ones
            return np.append(np.ones(n_ones, dtype=dtype), np.exp((1-k)*np.arange(n_other), dtype=dtype))
        if prof_type == "linear":
            profile = np.linspace(1.0,0.0, num=C, endpoint=False, dtype='float32')
        elif prof_type == "all-one":
            profile = np.ones(C, dtype='float32')
        elif prof_type == "half-exp":
            profile = half_exp(C, 2.0)
        elif prof_type == "harmonic":
            profile = np.array(1.0/(np.arange(C)+1))
        else:
            raise ValueError("prof_type must be \"all-one\", \"half-exp\", \"harmonic\" or \"linear\".")
        return profile
                

In [4]:
########################################################################
#
# Functions for downloading the CIFAR-10 data-set from the internet
# and loading it into memory.
#
# Implemented in Python 3.5
#
# Usage:
# 1) Set the variable data_path with the desired storage path.
# 2) Call maybe_download_and_extract() to download the data-set
#    if it is not already located in the given data_path.
# 3) Call load_class_names() to get an array of the class-names.
# 4) Call load_training_data() and load_test_data() to get
#    the images, class-numbers and one-hot encoded class-labels
#    for the training-set and test-set.
# 5) Use the returned data in your own program.
#
# Format:
# The images for the training- and test-sets are returned as 4-dim numpy
# arrays each with the shape: [image_number, height, width, channel]
# where the individual pixels are floats between 0.0 and 1.0.
#
########################################################################
#
# This file is part of the TensorFlow Tutorials available at:
#
# https://github.com/Hvass-Labs/TensorFlow-Tutorials
#
# Published under the MIT License. See the file LICENSE for details.
#
# Copyright 2016 by Magnus Erik Hvass Pedersen
#
########################################################################

import numpy as np
import pickle

class CIFAR10(object):
    def __init__(self):

        ########################################################################
        # Directory where you want to download and save the data-set.
        # Set this before you start calling any of the functions below.
        self.data_path = "/home/cmchang/IDP_CNN/data/"

        # URL for the data-set on the internet.
        self.data_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

        ########################################################################
        # Various constants for the size of the images.
        # Use these constants in your own program.

        # Width and height of each image.
        self.img_size = 32

        # Number of channels in each image, 3 channels: Red, Green, Blue.
        self.num_channels = 3

        # Length of an image when flattened to a 1-dim array.
        self.img_size_flat = self.img_size * self.img_size * self.num_channels

        # Number of classes.
        self.num_classes = 10

        ########################################################################

    def _one_hot_encoded(self, class_numbers, num_classes=None):
        """
        Generate the One-Hot encoded class-labels from an array of integers.
        For example, if class_number=2 and num_classes=4 then
        the one-hot encoded label is the float array: [0. 0. 1. 0.]
        :param class_numbers:
            Array of integers with class-numbers.
            Assume the integers are from zero to num_classes-1 inclusive.
        :param num_classes:
            Number of classes. If None then use max(class_numbers)+1.
        :return:
            2-dim array of shape: [len(class_numbers), num_classes]
        """

        # Find the number of classes if None is provided.
        # Assumes the lowest class-number is zero.
        if num_classes is None:
            num_classes = np.max(class_numbers) + 1

        return np.eye(num_classes, dtype=float)[class_numbers]

    ########################################################################
    # Private functions for downloading, unpacking and loading data-files.

    def _get_file_path(self, filename=""):
        """
        Return the full path of a data-file for the data-set.

        If filename=="" then return the directory of the files.
        """

        return os.path.join(self.data_path, "cifar-10-batches-py/", filename)


    def _unpickle(self, filename):
        """
        Unpickle the given file and return the data.

        Note that the appropriate dir-name is prepended the filename.
        """

        # Create full path for the file.
        file_path = self._get_file_path(filename)

        print("Loading data: " + file_path)

        with open(file_path, mode='rb') as file:
            # In Python 3.X it is important to set the encoding,
            # otherwise an exception is raised here.
            data = pickle.load(file, encoding='bytes')

        return data


    def _convert_images(self, raw):
        """
        Convert images from the CIFAR-10 format and
        return a 4-dim array with shape: [image_number, height, width, channel]
        where the pixels are floats between 0.0 and 1.0.
        """

        # Convert the raw images from the data-files to floating-points.
        raw_float = np.array(raw, dtype=float) / 255.0

        # Reshape the array to 4-dimensions.
        images = raw_float.reshape([-1, self.num_channels, self.img_size, self.img_size])

        # Reorder the indices of the array.
        images = images.transpose([0, 2, 3, 1])

        return images


    def _load_data(self, filename):
        """
        Load a pickled data-file from the CIFAR-10 data-set
        and return the converted images (see above) and the class-number
        for each image.
        """

        # Load the pickled data-file.
        data = self._unpickle(filename)

        # Get the raw images.
        raw_images = data[b'data']

        # Get the class-numbers for each image. Convert to numpy-array.
        cls = np.array(data[b'labels'])

        # Convert the images.
        images = self._convert_images(raw_images)

        return images, cls

    ########################################################################
    # Public functions that you may call to download the data-set from
    # the internet and load the data into memory.

    def load_class_names(self):
        """
        Load the names for the classes in the CIFAR-10 data-set.

        Returns a list with the names. Example: names[3] is the name
        associated with class-number 3.
        """

        # Load the class-names from the pickled file.
        raw = _unpickle(filename="batches.meta")[b'label_names']

        # Convert from binary strings.
        names = [x.decode('utf-8') for x in raw]

        return names


    def load_training_data(self):
        """
        Load all the training-data for the CIFAR-10 data-set.

        The data-set is split into 5 data-files which are merged here.

        Returns the images, class-numbers and one-hot encoded class-labels.
        """
        # Number of files for the training-set.
        _num_files_train = 5

        # Number of images for each batch-file in the training-set.
        _images_per_file = 10000

        # Total number of images in the training-set.
        # This is used to pre-allocate arrays for efficiency.
        _num_images_train = _num_files_train * _images_per_file
        
        
        # Pre-allocate the arrays for the images and class-numbers for efficiency.
        images = np.zeros(shape=[_num_images_train, self.img_size, self.img_size, self.num_channels], dtype=float)
        cls = np.zeros(shape=[_num_images_train], dtype=int)

        # Begin-index for the current batch.
        begin = 0

        # For each data-file.
        for i in range(_num_files_train):
            # Load the images and class-numbers from the data-file.
            images_batch, cls_batch = self._load_data(filename="data_batch_" + str(i + 1))

            # Number of images in this batch.
            num_images = len(images_batch)

            # End-index for the current batch.
            end = begin + num_images

            # Store the images into the array.
            images[begin:end, :] = images_batch

            # Store the class-numbers into the array.
            cls[begin:end] = cls_batch

            # The begin-index for the next batch is the current end-index.
            begin = end

        return images, self._one_hot_encoded(class_numbers=cls, num_classes=self.num_classes)


    def load_test_data(self):
        """
        Load all the test-data for the CIFAR-10 data-set.

        Returns the images, class-numbers and one-hot encoded class-labels.
        """

        images, cls = self._load_data(filename="test_batch")

        return images, self._one_hot_encoded(class_numbers=cls, num_classes=self.num_classes)

    ########################################################################

    ########################################################################
    #
    # Functions for downloading and extracting data-files from the internet.
    #
    # Implemented in Python 3.5
    #
    ########################################################################
    #
    # This file is part of the TensorFlow Tutorials available at:
    #
    # https://github.com/Hvass-Labs/TensorFlow-Tutorials
    #
    # Published under the MIT License. See the file LICENSE for details.
    #
    # Copyright 2016 by Magnus Erik Hvass Pedersen
    #
    ########################################################################

    import sys
    import os
    import urllib.request
    import tarfile
    import zipfile

    ########################################################################


    def _print_download_progress(self, count, block_size, total_size):
        """
        Function used for printing the download progress.
        Used as a call-back function in maybe_download_and_extract().
        """

        # Percentage completion.
        pct_complete = float(count * block_size) / total_size

        # Status-message. Note the \r which means the line should overwrite itself.
        msg = "\r- Download progress: {0:.1%}".format(pct_complete)

        # Print it.
        sys.stdout.write(msg)
        sys.stdout.flush()


    ########################################################################


    def maybe_download_and_extract(self, url, download_dir):
        """
        Download and extract the data if it doesn't already exist.
        Assumes the url is a tar-ball file.
        :param url:
            Internet URL for the tar-file to download.
            Example: "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
        :param download_dir:
            Directory where the downloaded file is saved.
            Example: "data/CIFAR-10/"
        :return:
            Nothing.
        """

        # Filename for saving the file downloaded from the internet.
        # Use the filename from the URL and add it to the download_dir.
        filename = url.split('/')[-1]
        file_path = os.path.join(download_dir, filename)

        # Check if the file already exists.
        # If it exists then we assume it has also been extracted,
        # otherwise we need to download and extract it now.
        if not os.path.exists(file_path):
            # Check if the download directory exists, otherwise create it.
            if not os.path.exists(download_dir):
                os.makedirs(download_dir)

            # Download the file from the internet.
            file_path, _ = urllib.request.urlretrieve(url=url,
                                                      filename=file_path,
                                                      reporthook=self._print_download_progress)

            print()
            print("Download finished. Extracting files.")

            if file_path.endswith(".zip"):
                # Unpack the zip-file.
                zipfile.ZipFile(file=file_path, mode="r").extractall(download_dir)
            elif file_path.endswith((".tar.gz", ".tgz")):
                # Unpack the tar-ball.
                tarfile.open(name=file_path, mode="r:gz").extractall(download_dir)

            print("Done.")
        else:
            print("Data has apparently already been downloaded and unpacked.")

    ########################################################################

In [5]:
def initialize_uninitialized(sess):
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v,f) in zip(global_vars, is_not_initialized) if not f]
    
    # print([str(i.name) for i in not_initialized_vars])
    
    if len(not_initialized_vars): 
            sess.run(tf.variables_initializer(not_initialized_vars))

In [6]:
import tensorflow as tf
import numpy as np
#import VGG16
#import CIFAR10

dataset = CIFAR10()
Xtrain, Ytrain = dataset.load_training_data()
Xtest, Ytest = dataset.load_test_data()

###
dp = [(i+1)*0.05 for i in range(1,20)]
vgg16 = VGG16("vgg16_weights.npz")
#vgg16 = VGG16("vgg16.npy")
vgg16.build(dp=dp, prof_type="all-one")

Loading data: /home/cmchang/IDP_CNN/data/cifar-10-batches-py/data_batch_1
Loading data: /home/cmchang/IDP_CNN/data/cifar-10-batches-py/data_batch_2
Loading data: /home/cmchang/IDP_CNN/data/cifar-10-batches-py/data_batch_3
Loading data: /home/cmchang/IDP_CNN/data/cifar-10-batches-py/data_batch_4
Loading data: /home/cmchang/IDP_CNN/data/cifar-10-batches-py/data_batch_5
Loading data: /home/cmchang/IDP_CNN/data/cifar-10-batches-py/test_batch
npy file loaded
Will optimize at DP= [0.1, 0.15000000000000002, 0.2, 0.25, 0.30000000000000004, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.65, 0.7000000000000001, 0.75, 0.8, 0.8500000000000001, 0.9, 0.9500000000000001, 1.0]
build model started


KeyError: 'conv1_1 is not a file in the archive'

### Unit test 

In [ ]:
# hyper parameters
learning_rate = 1e-3
batch_size = 200
alpha = 0.5
early_stop_patience = 4

In [ ]:
obj = vgg16.loss_dict['100']
# optimizer
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(obj)
train_op = tf.train.GradientDescentOptimizer()

with tf.Session() as sess:
    
    summary_writer = tf.summary.FileWriter("log/",sess.graph)
    
    sess.run(tf.global_variables_initializer())
    val_loss = sess.run(vgg16.accu_dict['100'],feed_dict={vgg16.x: Xtest[:5000,:,:,:],
                                   vgg16.y: Ytest[:5000,:]})
    print("Predicted model: {a:.4f}".format(a=val_loss))
    
    # training an epoch
    for i in range(5):
        st = i*batch_size
        ed = (i+1)*batch_size
        _, summary = sess.run([train_op, vgg16.summary_op], feed_dict={vgg16.x: Xtrain[st:ed,:,:,:],
                                        vgg16.y: Ytrain[st:ed,:]})
        summary_writer.add_summary(summary, i)
#         # print(summary)
#     val_loss = sess.run(vgg16.accu_dict['100'],feed_dict={vgg16.x: Xtest[:5000,:,:,:],
#                                    vgg16.y: Ytest[:5000,:]})
#     print("Predicted model: {a:.4f}".format(a=val_loss))
    
#     val_loss = sess.run(vgg16.accu_dict['70'],feed_dict={vgg16.x: Xtest[:5000,:,:,:],
#                                    vgg16.y: Ytest[:5000,:]})
#     print("Predicted model: {a:.4f}".format(a=val_loss))
    
#     val_loss = sess.run(vgg16.accu_dict['30'],feed_dict={vgg16.x: Xtest[:5000,:,:,:],
#                                    vgg16.y: Ytest[:5000,:]})
#     print("Predicted model: {a:.4f}".format(a=val_loss))
    summary_writer.close()

### TESLA

In [ ]:
tasks = sorted([int(dp_i*100) for dp_i in dp],reverse=True)
tasks = [str(task) for task in tasks]

In [ ]:
# idx = np.random.permutation(Xtest.shape[0])[:(int(Xtest.shape[0]/2))]
# Xtest, Ytest= Xtest[idx,:], Ytest[idx,:]

In [13]:
saver = tf.train.Saver(tf.global_variables())
tasks = ['100', '70', '40', '10']
# initial task
obj = vgg16.loss_dict[tasks[0]]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # hyper parameters
    learning_rate = 2e-4
    batch_size = 32
    alpha = 0.5
    early_stop_patience = 4
    min_delta = 0.0001
    
    # optimizer
    #opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
    opt = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    
    # recorder
    epoch_counter = 0
    
    # tensorboard writer
    writer = tf.summary.FileWriter("log/", sess.graph)
    
    # progress bar
    from progress.bar import Bar
    
    from ipywidgets import IntProgress
    from IPython.display import display
    ptrain = IntProgress()
    pval = IntProgress()
    display(ptrain)
    display(pval)
    ptrain.max = int(Xtrain.shape[0]/batch_size)
    pval.max = int(Xtest.shape[0]/batch_size)
    
    while(len(tasks)):
       
        # acquire a new task
        cur_task = tasks[0]
        tasks = tasks[1:]
        new_obj = vgg16.loss_dict[cur_task]
        
        # task-wise loss aggregation
        obj = tf.add(tf.multiply(obj, 1.0-alpha), tf.multiply(new_obj, alpha))
        
        # optimizer
        train_op = opt.minimize(obj)
        
        # re-initialize
        initialize_uninitialized(sess)
        
        # reset due to adding a new task
        patience_counter = 0
        current_best_val_loss = 100000 # a large number
        
        # optimize when the aggregated obj
        while(patience_counter < early_stop_patience):
            stime = time.time()
            bar_train = Bar('Training', max=int(Xtrain.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
            bar_val =  Bar('Validation', max=int(Xtest.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
            # training an epoch
            for i in range(int(Xtrain.shape[0]/batch_size)):
                st = i*batch_size
                ed = (i+1)*batch_size
                sess.run([train_op], feed_dict={vgg16.x: Xtrain[st:ed,:,:,:],
                                                vgg16.y: Ytrain[st:ed,:]})
                ptrain.value +=1
                ptrain.description = "Training %s/%s" % (i, ptrain.max)
                bar_train.next()
            
            # validation
            val_loss = 0
            val_accu = 0
            for i in range(int(Xtest.shape[0]/batch_size)):
                st = i*batch_size
                ed = (i+1)*batch_size
                loss, accu, epoch_summary = sess.run([obj, vgg16.accu_dict[cur_task], vgg16.summary_op],
                                    feed_dict={vgg16.x: Xtest[st:ed,:],
                                               vgg16.y: Ytest[st:ed,:]})
                val_loss += loss
                val_accu += accu
                pval.value += 1
                pval.description = "Testing %s/%s" % (i, pval.max)
            #val_loss = np.mean(val_loss,axis=-1)
            #val_accu = np.mean(val_accu,axis=-1)
            val_loss = val_loss/pval.max
            val_accu = val_accu/pval.max
            
            # early stopping check
            if (current_best_val_loss - val_loss) > min_delta:
                current_best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1
            
            # shuffle Xtrain and Ytrain in the next epoch
            idx = np.random.permutation(Xtrain.shape[0])
            Xtrain, Ytrain = Xtrain[idx,:,:,:], Ytrain[idx,:]
            

#             epoch_summary = sess.run([vgg16.summary_op],
#                                     feed_dict={vgg16.x: Xtest,
#                                                vgg16.y: Ytest})
            
            # epoch end
            writer.add_summary(epoch_summary, epoch_counter)
            epoch_counter += 1
            
            ptrain.value = 0
            pval.value = 0
            bar_train.finish()
            bar_val.finish()
            
            print("Epoch %s (%s), %s sec >> obj loss: %.4f, task at %s: %.4f" % (epoch_counter, patience_counter, round(time.time()-stime,2), val_loss, cur_task, val_accu))

    writer.close()

Epoch 1 (0), 78.7 sec >> obj loss: 0.8100, task at 100: 0.7217
Epoch 2 (0), 78.28 sec >> obj loss: 0.6649, task at 100: 0.7732
Epoch 3 (0), 78.65 sec >> obj loss: 0.5973, task at 100: 0.7961
Epoch 4 (0), 78.59 sec >> obj loss: 0.5464, task at 100: 0.8155
Epoch 5 (0), 78.08 sec >> obj loss: 0.5087, task at 100: 0.8330
Epoch 6 (1), 78.47 sec >> obj loss: 0.5217, task at 100: 0.8277
Epoch 7 (0), 78.58 sec >> obj loss: 0.4970, task at 100: 0.8417
Epoch 8 (0), 78.3 sec >> obj loss: 0.4895, task at 100: 0.8428
Epoch 9 (1), 78.46 sec >> obj loss: 0.5340, task at 100: 0.8340
Epoch 10 (2), 78.45 sec >> obj loss: 0.5117, task at 100: 0.8435
Epoch 11 (3), 78.35 sec >> obj loss: 0.5248, task at 100: 0.8476
Epoch 12 (4), 78.47 sec >> obj loss: 0.5334, task at 100: 0.8493
Epoch 13 (0), 112.75 sec >> obj loss: 0.7562, task at 70: 0.6721
Epoch 14 (0), 112.76 sec >> obj loss: 0.6974, task at 70: 0.7357
Epoch 15 (0), 112.69 sec >> obj loss: 0.6612, task at 70: 0.7669
Epoch 16 (1), 112.72 sec >> obj loss

In [18]:
saver = tf.train.Saver(tf.global_variables())
checkpoint_path = os.path.join('save/', 'model.ckpt')

tasks = ['100', '70', '40', '10']
# initial task
obj = vgg16.loss_dict[tasks[0]]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # hyper parameters
    learning_rate = 2e-4
    batch_size = 32
    alpha = 0.5
    early_stop_patience = 4
    min_delta = 0.0001
    
    # optimizer
    #opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
    opt = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    
    # recorder
    epoch_counter = 0
    
    # tensorboard writer
    writer = tf.summary.FileWriter("log/", sess.graph)
    
    # progress bar
    from progress.bar import Bar
    
    from ipywidgets import IntProgress
    from IPython.display import display
    ptrain = IntProgress()
    pval = IntProgress()
    display(ptrain)
    display(pval)
    ptrain.max = int(Xtrain.shape[0]/batch_size)
    pval.max = int(Xtest.shape[0]/batch_size)
    
    while(len(tasks)):
       
        # acquire a new task
        cur_task = tasks[0]
        tasks = tasks[1:]
        new_obj = vgg16.loss_dict[cur_task]
        
        # task-wise loss aggregation
        obj = tf.add(tf.multiply(obj, 1.0-alpha), tf.multiply(new_obj, alpha))
        
        # optimizer
        train_op = opt.minimize(obj)
        
        # re-initialize
        initialize_uninitialized(sess)
        
        # reset due to adding a new task
        patience_counter = 0
        current_best_val_loss = 100000 # a large number
        
        # optimize when the aggregated obj
        while(patience_counter < early_stop_patience):
            stime = time.time()
            bar_train = Bar('Training', max=int(Xtrain.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
            bar_val =  Bar('Validation', max=int(Xtest.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
            # training an epoch
            for i in range(int(Xtrain.shape[0]/batch_size)):
                st = i*batch_size
                ed = (i+1)*batch_size
                sess.run([train_op], feed_dict={vgg16.x: Xtrain[st:ed,:,:,:],
                                                vgg16.y: Ytrain[st:ed,:]})
                ptrain.value +=1
                ptrain.description = "Training %s/%s" % (i, ptrain.max)
                bar_train.next()
            
            # validation
            val_loss = 0
            val_accu = 0
            for i in range(int(Xtest.shape[0]/200)):
                st = i*200
                ed = (i+1)*200
                loss, accu, epoch_summary = sess.run([obj, vgg16.accu_dict[cur_task], vgg16.summary_op],
                                    feed_dict={vgg16.x: Xtest[st:ed,:],
                                               vgg16.y: Ytest[st:ed,:]})
                val_loss += loss
                val_accu += accu
                pval.value += 1
                pval.description = "Testing %s/%s" % (i, pval.max)
            #val_loss = np.mean(val_loss,axis=-1)
            #val_accu = np.mean(val_accu,axis=-1)
            val_loss = val_loss/pval.max
            val_accu = val_accu/pval.max
            
            # early stopping check
            if (current_best_val_loss - val_loss) > min_delta:
                current_best_val_loss = val_loss
                patience_counter = 0
                saver.save(sess, checkpoint_path, global_step=epoch_counter)
            else:
                patience_counter += 1
            
            # shuffle Xtrain and Ytrain in the next epoch
            idx = np.random.permutation(Xtrain.shape[0])
            Xtrain, Ytrain = Xtrain[idx,:,:,:], Ytrain[idx,:]
            

#             epoch_summary = sess.run([vgg16.summary_op],
#                                     feed_dict={vgg16.x: Xtest,
#                                                vgg16.y: Ytest})
            
            # epoch end
            writer.add_summary(epoch_summary, epoch_counter)
            epoch_counter += 1
            
            ptrain.value = 0
            pval.value = 0
            bar_train.finish()
            bar_val.finish()
            
            print("Epoch %s (%s), %s sec >> obj loss: %.4f, task at %s: %.4f" % (epoch_counter, patience_counter, round(time.time()-stime,2), val_loss, cur_task, val_accu))

    writer.close()

Epoch 1 (0), 61.73 sec >> obj loss: 0.1561, task at 100: 0.1100
Epoch 2 (0), 60.05 sec >> obj loss: 0.1114, task at 100: 0.1230
Epoch 3 (0), 59.91 sec >> obj loss: 0.0935, task at 100: 0.1283


KeyboardInterrupt: 

In [14]:
import pandas as pd

accu_log = []
loss_log = []
tag_log = []
counter = 0
for e in tf.train.summary_iterator("log/events.out.tfevents.1516590873.theta1"):
    counter += 1
    for v in e.summary.value:
        if 'accu' in v.tag:
            tag_log.append(str(counter-3)+"/"+v.tag)
            accu_log.append(v.simple_value)
        if 'loss' in v.tag:
            loss_log.append(v.simple_value)

output = pd.DataFrame.from_dict({'tag':tag_log, 'loss':loss_log, 'accu':accu_log})
output.to_csv("output.csv", index=None)